# SunCorp
The [Suncorp Broker Portal](https://brokerportal.suncorpbank.com.au/home/policy/retail.html) is a single page HTML.
1. Policy heading and text is inside div class Main-content. 
2. Uses H1-3 H3. 
3. Uses <button id="parental-leave" type="button" class="sg-Accordion-label js-Accordion is-selected" aria-controls="Content-parental-leave" role="tab" data-event-type="Accordion" data-event-details="Accordion" aria-selected="true">Parental Leave</button> for H3 or 4?

The policy is wrtten to policy/Suncorp/Broker_Portal

In [6]:
import requests
from bs4 import BeautifulSoup
import shutil
import os

In [20]:
from  scraper import WebScraper
from loguru import logger
import os 

class Suncorp(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["main.Main"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True) -> None:

        logger.info(f"Scraping {depth}: {url}")
        
        if self.skip(url, depth, exclude=["tool", "help-and-support"]):
            return
             
        _, extension = self.get_path(url)
        assert extension in [".html"], f"Invalid extension '{extension}' for {url}"

        html_content = self.fetch_content(url) 
        html_content = self.remove_script(html_content) 
        if not (starting_page or self.search_only):
            self.save_content(url, html_content)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "main.Main")

# Download Suncorp

In [23]:
import shutil

directory_path=r"./policy/Suncorp/Broker_Portal"
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
suncorp = Suncorp("https://brokerportal.suncorpbank.com.au", 
              cookie="""visid_incap_2935922=s+FqZZBQTLy9vetpTaw6wdP03WcAAAAAQUIPAAAAAAAiMbG37iJmu7xOBP8xnWjP; nlbi_2935922=0nj9VlEaV2D+8KFX2Jhg2wAAAAA+gTIzfw4ADHgcfJN7GtQm; OAM_REQ_COUNT=VERSION_4~1; cookie-encrypt=!tm0mG/LOQUA+3l7/9oA6fj8w6p/B3/ELO6TacCLg5VoDEiqOxLKD1OkWdjYAzlsAoVTM75tn4Udrew==; a19cafdefa3a1f020d895050bc5faf97=4637c6f372bb951c40ab4e1f0845b6ac; said=B195BB045D96807448583C97D0909B32; at_check=true; 4304=; 64542=B195BB045D96807448583C97D0909B32; s_ecid=MCMID%7C89508307447949345274997596830337074768; AMCVS_223234B85278553C0A490D44%40AdobeOrg=1; s_traffictype_b=0; s_serialization_id=1742599380872.6526; s_cc=true; _ga=GA1.1.42037934.1742599381; affinity="d60eaf12b198f862"; visid_incap_3079665=iKu9CYqySzimvXmAp3Uk/Uz13WcAAAAAQUIPAAAAAAAS6yVV6SZYCyivjHhhGZ7D; _gcl_au=1.1.1116724274.1742599505; _fbp=fb.2.1742599505561.60901821508639100; s_nr=1742601108491; s_traffictype_ql=%20; visid_incap_2921568=ClIFycPmRI6Gda3MCkWLGLFe3mcAAAAAQUIPAAAAAAD3fPLslxq4WK2NRY7NDQLe; nlbi_2921568=QAePJpgEpi4aK+NRfS+TLgAAAAC5KsEi3FN6jnC/aN2W8Vrq; incap_ses_361_2921568=mXNoPsA0YUu2ecGV14cCBbFe3mcAAAAAqrNbO4rXTEhJRQkR0m4Flw==; incap_ses_361_3079665=OxIFBVot6xQVYCyW14cCBb6Z32cAAAAAkRerSMbZWDok3eQvKoPSeA==; OAM_JSESSIONID=FwTCHBzeHiq-YApL8CwumBnbRTh6_sh6QURiThMy-_wexNKnGVf5!-1385049167; incap_ses_361_2935922=WBX8FnkwYFVvs5eW14cCBXO64GcAAAAA0Kq6IpTng0ecMhVUNoUaCA==; s_serialization_uuid=76d85a55a90b666b1c9e; AMCV_223234B85278553C0A490D44%40AdobeOrg=-2078930370%7CMCIDTS%7C20171%7CMCMID%7C89508307447949345274997596830337074768%7CMCAID%7CNONE%7CMCOPTOUT-1742788249s%7CNONE%7CvVersion%7C5.5.0%7CMCAAMLH-1743385849%7C8%7CMCAAMB-1743385849%7Cj8Odv6LonN4r3an7LhD3WZrU1bUpAkFkkiY1ncBR96t2PTI; s_vnum=1743436800825%26vn%3D10; s_invisit=true; s_traffictype_ss=0x0x0x10; TS013793fd=01d1b2ebf01a764192fa9ec7e6c6b045e3c58b6beb7575fe28e70f3670a0ad57bf3e1ff803ecb89c56d59a4eb999b4bab81c5dc8a2; OAM_ID=VERSION_4~NShcnQItzazzcpt/32cKfA==~WybwMl8hvEhbKHU2IaUvhOkNNp7GmVx8z7ZdYSv1ezRNvFSAJaX4X/G49TKWFZYnHXLTuGfu9b8ATizAa0v2FQBZkUAxZUU+xjYcr3ey54q0HYT/WHqurfsKluP+2TMiKjV/AAXhZMcCiz2iOxRhYuqdK/SmKr3uEdWa3YmEMM5KQbIDJ4HXtuq932aGYG9fs8sj43kAl+e7l8UFOVFVt8nvN9sLH9h912OO4wKVu7YrHE3PlkAbgWnclc/4KBsNngd0F3R48EEvNX5rhQGSq/o803318GsHJ5SNnWWdhVv91IG+TuawD24XtbUTbSsyv/Lxg6yNx1UwZMIhwXeD4Q==; OAM_REQ_0=invalid; OAMAuthnCookie_brokerportal.suncorpbank.com.au:443=1e9f67e6bb8eb1434432471e128a689e1c584a63%7EV%2ByxfFTqVxpqzKl5tKqMwMxIZ6nQ4jx7cRLq76QbTqnIYqwR30Hp8P0x3XMzXmPmCWLrvHhU%2FxSc2V4ll57jAscQqbsSLKe4RNua%2FmhSIR9iuJkok1EOIke9g6%2BeMC2z%2B2Ja6qHxQXkW6tbH9v9nOuY1NJlCPhOzpwXjaqLYBHXDCMuFQA2edJKuQz5jPqxc3uXJRqht%2FSYHJ%2FSTfr3mnicpvhwza1zrGlTsi2rcG%2FjpzJEb0cQqWN0msQ6JwZBAHnPt%2FmmVz3Xjguc67YZR4tkJ%2FFK8I0w58n7azJNX3vjD2HIg9Fs96KyT4aCkzAUxL%2BHeDGjlYMy6EsUZk5I9rMWCCj6LP2vEdnOfCCeorkIl6rJzLLXSCxzOGkW6iUc4ddxxntlbNkKj1GmzRqKe13IltqsOG7me72XYFmQH1homxreNSynHviwVyrDK75PBoPoWFgSYE0t%2BcwafRJydUe1T4UUxI%2B%2F9WcJFabYcKVGysCwRBuq%2B%2FKH%2BfUp0J%2F%2FkXQ7mrYpCgtF4VtHXNHFH0qm4VLYx8pByOOCXZY%2FyPnUfD3lPBsiSwN41aUsB9X%2FpVTI3vJL%2F6C3vP9ZFddhj%2Bw%3D%3D; mbox=PC#ad6e47c3f9ef4d9292db2bca0a9afb16.36_0#1806027354|session#62e86324d4c0405195744a9f3024412d#1742784414; _ga_M687NX67DN=GS1.1.1742781047.12.1.1742782554.53.0.0; s_sq=sunbkmasterprod%3D%2526c.%2526a.%2526activitymap.%2526page%253Dbk%25253Asun%25253Ahome%25253Apolicy%25253Aretail%2526link%253DIncome%2526region%253DBODY%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c""", 
              download_path= directory_path, 
              search_only=False)
suncorp.scrape("https://brokerportal.suncorpbank.com.au/home/policy/retail.html", depth=16)

print(len(suncorp.download))

2025-03-24 10:34:28.214 | INFO     | __main__:scrape:12 - Scraping 16: https://brokerportal.suncorpbank.com.au/home/policy/retail.html
2025-03-24 10:34:28.736 | INFO     | __main__:scrape:12 - Scraping 15: https://brokerportal.suncorpbank.com.au/home.html
2025-03-24 10:34:29.082 | INFO     | scraper:save_content:56 - Downloading https://brokerportal.suncorpbank.com.au/home.html to ./policy/Suncorp/Broker_Portal/home.html
2025-03-24 10:34:29.143 | INFO     | __main__:scrape:12 - Scraping 14: https://brokerportal.suncorpbank.com.au/home/service-levels.html
2025-03-24 10:34:29.451 | INFO     | scraper:save_content:56 - Downloading https://brokerportal.suncorpbank.com.au/home/service-levels.html to ./policy/Suncorp/Broker_Portal/home/service-levels.html
2025-03-24 10:34:29.491 | INFO     | __main__:scrape:12 - Scraping 13: https://brokerportal.suncorpbank.com.au/home.html
2025-03-24 10:34:29.492 | INFO     | __main__:scrape:12 - Scraping 13: https://brokerportal.suncorpbank.com.au/home/ser

32
